In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/_scripts")

from pint import UnitRegistry

from svg_helpers.saver import Saver, read_layout
from svg_helpers.plotter import Plotter

from svg_helpers.domains import Corners, DomainDict, Domain, DecimalCorners

from reader.svg_reader import SVGReader
from adjacencies.adjacency import AdjacencyGenerator
from adjacencies.connectivity import ConnectivityGenerator

from placement.executer import PlacementExecuter


from problems.sequence_runner import SequenceRunner
from problems.classes.sequence import Sequence
from problems.reporter import Reporter

from svg_helpers.gplan_creator import GPLANCreator

from problems.side_leveler import SideLeveler

from decimal import Decimal, getcontext, Context

from pprint import pprint

In [2]:
h = Context(prec=4)
Decimal(1.4567890, h) + Decimal(1.4567890, h)

Decimal('2.913577999999999779134896016')

In [3]:
svg_name = "amber_building_a_f01.svg"
sv = SVGReader(svg_name)
# sv.get_rectangles()
# sv.prepare_to_update_dimensions()
sv.run()

In [4]:
pl = Plotter(sv.domains.corners)
pl.plot()

In [5]:
ag = AdjacencyGenerator(sv.domains, 0.4)
ag.run()

In [6]:
ag.layout_obj

Layout(shapes={'bedroom_1': <POLYGON ((3.22 3.29, 3.22 6.09, 0 6.09, 0 3.29, 3.22 3.29))>, 'kitchen': <POLYGON ((5.16 0, 5.16 2.24, 3.36 2.24, 3.36 0, 5.16 0))>, 'm_bath': <POLYGON ((7.62 4.79, 7.62 6.09, 5.3 6.09, 5.3 4.79, 7.62 4.79))>, 'm_bedroom': <POLYGON ((9.93 3.35, 9.93 6.09, 7.8 6.09, 7.8 3.35, 9.93 3.35))>, 'm_closet': <POLYGON ((9.93 2.69, 9.93 3.29, 8.93 3.29, 8.93 2.69, 9.93 2.69))>, 'bath': <POLYGON ((7.62 3.35, 7.62 4.58, 5.3 4.58, 5.3 3.35, 7.62 3.35))>, 'dining': <POLYGON ((8.78 0, 8.78 3.15, 5.42 3.15, 5.42 0, 8.78 0))>, 'transit_space': <POLYGON ((5.16 2.41, 5.16 4.19, 3.36 4.19, 3.36 2.41, 5.16 2.41))>, 'closet': <POLYGON ((9.93 -0.05, 9.93 2.36, 8.93 2.36, 8.93 -0.05, 9.93 -0.05))>, 'bedroom_2': <POLYGON ((3.22 0, 3.22 3.15, 0 3.15, 0 0, 3.22 0))>, 'laundry': <POLYGON ((4.98 4.31, 4.98 6.06, 3.38 6.06, 3.38 4.31, 4.98 4.31))>}, corners={'bedroom_1': DecimalCorners(x_left=Decimal('0.00'), x_right=Decimal('3.22'), y_bottom=Decimal('3.29'), y_top=Decimal('6.09')), 'ki

In [7]:
ag.layout_obj.corners["m_closet"].x_left

Decimal('8.93')

In [8]:
cg = ConnectivityGenerator(ag.positioned_graph)
cg.run()

In [9]:
lp = PlacementExecuter(ag.layout_obj)
lp.run()

---m_bedroom has no western nbs that are unplaced
fpr bedroom_2, there is no west node
dining, the east node of m_closet has not yet been placed. Creating a new x_left at 8.20
changing north node reference to 1. Number of unplaced nodes is 3
no more nodes to place


In [10]:
re = Reporter(lp.layout)
print(re.problems)
re.run()
re.problems

[]
-- Unresolved Problems. Overlaps: 3. Holes: 1. Sideholes: 0


[Problem(index=1, problem_type=ProblemType.HOLE, resolved=False, nbs=['transit_space', 'laundry', 'm_bath', 'bath'], x_left=4.82),
 Problem(index=2, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining'), x_left=8.2),
 Problem(index=3, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'dining'), x_left=7.14),
 Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'bath'), x_left=7.14)]

In [12]:
seq = Sequence(0, lp.layout, re.problems, [])
# sr = SequenceRunner(seq, re.problems[0])
# sr.execute_actions()

In [13]:
sr = SequenceRunner(seq, re.problems[0])
sr.run()

--iteration #1
curr problem = Problem(index=1, problem_type=ProblemType.HOLE, resolved=False, nbs=['transit_space', 'laundry', 'm_bath', 'bath'], x_left=4.82)
Action(problem=((1, 'HOLE')), action_type=ActionType.STRETCH, node=laundry, distance=0.20)
-- Unresolved Problems. Overlaps: 4. Holes: 0. Sideholes: 0
problem resolved
indices: [2, 3, 4, 5]
next problem = Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'bath'), x_left=7.14)
--iteration #2
curr problem = Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'bath'), x_left=7.14)
Action(problem=((4, 'OVERLAP')), action_type=ActionType.PUSH, node=m_bedroom, distance=0.20)
-- Unresolved Problems. Overlaps: 3. Holes: 0. Sideholes: 1
problem resolved
indices: [2, 5, 6, 7]
next problem = Problem(index=5, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bath', 'laundry'), x_left=4.82)
--iteration #3
curr problem = Problem(index=5, problem_type=ProblemType.OVERLA

In [23]:
sr.execute_actions()

pl = Plotter(sr.layout.corners, yrange=[-9,0])
pl.plot()

--iteration #13
curr problem = Problem(index=13, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining'), x_left=8.38)
Action(problem=((13, 'OVERLAP')), action_type=ActionType.PUSH, node=m_closet, distance=0.20)
-- Unresolved Problems. Overlaps: 0. Holes: 0. Sideholes: 0
problem resolved
indices: []
no more problems!


In [27]:
sl = SideLeveler(sr.layout)
sl.run()

In [28]:
pl = Plotter(sl.layout.corners, yrange=[-9,0])
pl.plot()

In [30]:
sa = Saver(sl.layout, "correct_dims")
sa.save()